Ön hazırlık (kütüphane kurulumu)

In [1]:
# Gerekli kütüphaneleri yükleyelim (Colab hücresine)
!pip install cirq qiskit qutip numpy matplotlib --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 33.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 670.8/670.8 kB 39.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.5/73.5 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 430.5/430.5 kB 27.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.0/8.0 MB 83.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.8/31.8 MB 30.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 78.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 83.8 MB/s eta 0:00:00


1. Cirq ile Kuantum Durumlarının İkili Gösterimi ve QFT

In [2]:
import cirq
import numpy as np

# 2-qubit sistemi tanımla
qubits = cirq.LineQubit.range(2)
circuit = cirq.Circuit()

# Başlangıç durumu: |01⟩
circuit.append(cirq.X(qubits[0]))

print("Başlangıç durumu: |01⟩")
print(circuit)

# Kuantum Fourier Dönüşümü (QFT)
def qft_cirq(qubits):
    n = len(qubits)
    circuit = cirq.Circuit()
    for i in range(n):
        circuit.append(cirq.H(qubits[i]))
        for j in range(i+1, n):
            angle = np.pi / 2**(j - i)
            circuit.append(cirq.CZ(qubits[j], qubits[i])**(angle / np.pi))
    circuit.append(cirq.SWAP(qubits[0], qubits[1]))
    return circuit

qft_circuit = qft_cirq(qubits)
full_circuit = circuit + qft_circuit
print("\nQFT devresi:")
print(full_circuit)

# Simülasyon
sim = cirq.Simulator()
result = sim.simulate(full_circuit)
print("\nSon durum vektörü (QFT sonrası):")
print(np.round(result.final_state_vector, 3))


Başlangıç durumu: |01⟩
0: ───X───

QFT devresi:
0: ───X───H───@───────────×───
              │           │
1: ───────────@^0.5───H───×───

Son durum vektörü (QFT sonrası):
[ 0.5+0.j -0.5+0.j  0.5+0.j -0.5+0.j]


2. Qiskit ile Kuantum Durumlarının İkili Gösterimi ve QFT

In [3]:
from qiskit import QuantumCircuit, Aer, execute
from qiskit.circuit.library import QFT
import numpy as np

# 2-qubit devre
qc = QuantumCircuit(2)

# Başlangıç: |10⟩
qc.x(1)

# QFT uygula
qc.append(QFT(2, do_swaps=True), [0,1])

qc.draw('mpl')

# Simülasyon
backend = Aer.get_backend('statevector_simulator')
result = execute(qc, backend).result()
state = result.get_statevector()

print("QFT sonrası durum vektörü:")
print(np.round(state, 3))


ImportError: cannot import name 'Aer' from 'qiskit' (/usr/local/lib/python3.12/dist-packages/qiskit/__init__.py)

3. QuTiP ile Kuantum Durumlarının İkili Gösterimi ve QFT

In [4]:
from qutip import *
import numpy as np

# 2-qubit |01⟩ durumu
psi = tensor(basis(2,0), basis(2,1))
print("Başlangıç durumu |01⟩:\n", psi.full())

# QFT operatörünü tanımla
def qft_operator(n):
    N = 2**n
    omega = np.exp(2j * np.pi / N)
    U = np.zeros((N, N), dtype=complex)
    for j in range(N):
        for k in range(N):
            U[j, k] = omega**(j * k) / np.sqrt(N)
    return Qobj(U)

# 2-qubit QFT
U_qft = qft_operator(2)
psi_qft = U_qft * psi

print("\nQFT sonrası durum vektörü:")
print(np.round(psi_qft.full(), 3))


Başlangıç durumu |01⟩:
 [[0.+0.j]
 [1.+0.j]
 [0.+0.j]
 [0.+0.j]]


TypeError: incompatible dimensions [[4], [4]] and [[2, 2], [1]]